In [1]:
from __future__ import annotations

import os

import mgz.settings as settings

os.putenv("PYTORCH_ENABLE_MPS_FALLBACK", "1")
# from mgz.models.nlp.bart_interface import BARTHubInterface

from mgz.typing import *
import transformers as hug

CUDA_LAUNCH_BLOCKING 1
1 gpus available, will be using gpu 0


In [2]:


logging.basicConfig(level=logging.WARNING)
model_name = 'openchat/openchat_3.5'
print('... loading model and tokenizer')
with torch.cuda.amp.autocast(enabled=True):
    quantize = True
    quantization_cfg = None
    if quantize:
        try:
            from accelerate.utils import BnbQuantizationConfig
            import bitsandbytes

            quantization_cfg = BnbQuantizationConfig(
                load_in_8bit=quantize, )
        except ImportError:
            print("Module 'some_module' is not installed.")
            quantization_cfg = None
            quantize = False

    # model_node: ModelNode = \
    #     ModelNode.load_from_id(MistralForCausalLM, model_name,
    #                            model_name,
    #                            quantization_config=quantization_cfg)
    # tokenizer = model_node.tokenizer
    # model_node.model.eval()

    tokenizer = hug.LlamaTokenizerFast.from_pretrained(
        model_name)
    model_hug = hug.MistralForCausalLM.from_pretrained(model_name, device_map={
        "": settings.DEVICE},
                                                       load_in_8bit=True)
    model_hug.eval()


... loading model and tokenizer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ceyer/miniconda3/envs/Maghz/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [3]:
with (torch.cuda.amp.autocast(enabled=True)):
    tag_qa_text = (
        f"GPT4 Correct User: Is this e-mail about company business or strategy?: "
        """
        Message-ID: <23743848.1075863311776.JavaMail.evans@thyme>
        Date: Wed, 11 Jul 2001 08:29:22 -0700 (PDT)
        From: legalonline-compliance@enron.com
        To: williams@mailman.enron.com, bwillia5@enron.com
        Subject: Confidential Information and Securities Trading
        Mime-Version: 1.0
        Content-Type: text/plain; charset=us-ascii
        Content-Transfer-Encoding: 7bit
        X-From: Office of the Chairman - Enron Wholesale Services <legalonline-compliance@enron.com>@ENRON <IMCEANOTES-Office+20of+20the+20Chairman+20-+20Enron+20Wholesale+20Services+20+3Clegalonline-compliance+40enron+2Ecom+3E+40ENRON@ENRON.com>
        X-To: WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@enron.com>
        X-cc: 
        X-bcc: 
        X-Folder: \Williams III, Bill (Non-Privileged)\Bill Williams III
        X-Origin: Williams-B
        X-FileName: Williams III, Bill (Non-Privileged).pst
        
        To:WILLIAMS, WILLIAM
        Email:bwillia5@enron.com - 503-464-3730
        
        Enron Wholesale Services - Office of the Chairman
        
        From:  Mark Frevert, Chairman & CEO
        Mark Haedicke, Managing Director & General Counsel
        
        Subject:  Confidential Information and Securities Trading
        
        To keep pace with the fluid and fast-changing demands of our equity trading activities, Enron Wholesale Services ("EWS") has recently revised its official Policies and Procedures Regarding Confidential Information and Securities Trading ("Policies and Procedures").  These revisions reflect two major developments: (1) our equity trading activities have been extended into the United Kingdom, and (2) in an effort to streamline the information flow process, the "Review Team" will play a more centralized role, so that the role of the "Resource Group" is no longer necessary.You are required to become familiar with, and to comply with, the Policies and Procedures.  The newly revised Policies and Procedures are available for your review on LegalOnline, the new intranet website maintained by the Enron Wholesale Services Legal Department.  Please click on the attached link to access LegalOnline:
        http://legalonline.corp.enron.com/chinesewall.asp 
        
        If you have already certified compliance with the Policies and Procedures during the 2001 calendar year, you need not re-certify at this time, although you are still required to to review and become familiar with the revised Policies and Procedures.  If you have not certified compliance with the Policies and Procedures during the 2001 calendar year, then you must do so within two weeks of your receipt of this message.  The LegalOnline site will allow you to quickly and conveniently certify your compliance on-line with your SAP Personal ID number.  If you have any questions concerning the Policies or Procedures, please call Bob Bruce at extension 5-7780 or Donna Lowry at extension 3-1939. 
        """
        f"<|end_of_turn|>GPT4 Correct Assistant: ")
    text = [tag_qa_text]

    # tokens = generate_controller(model_node.model, text,
    #                              model_node.tokenizer)
    # summary: List[str] = model_node.tokenizer.batch_decode(tokens,
    #                                                        skip_special_tokens=True)
    # print('mgz decoded', summary)

    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenized = tokenizer(text, return_tensors='pt',
                          padding=True)
    input_ids = tokenized.input_ids.to(settings.DEVICE)
    attention_mask = tokenized.attention_mask.to(settings.DEVICE)
    generated_ids = model_hug.generate(input_ids, attention_mask=attention_mask,
                                       max_new_tokens=1000)
    # print('generated_ids', generated_ids)
    summary: List[str] = tokenizer.batch_decode(generated_ids,
                                                skip_special_tokens=True)
    print('hug decoded', summary)



/home/ceyer/miniconda3/envs/Maghz/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
2023-12-23 20:26:26.093106: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-23 20:26:26.093120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


hug decoded ['GPT4 Correct User: Is this e-mail about company business or strategy?: \n        Message-ID: <23743848.1075863311776.JavaMail.evans@thyme>\n        Date: Wed, 11 Jul 2001 08:29:22 -0700 (PDT)\n        From: legalonline-compliance@enron.com\n        To: williams@mailman.enron.com, bwillia5@enron.com\n        Subject: Confidential Information and Securities Trading\n        Mime-Version: 1.0\n        Content-Type: text/plain; charset=us-ascii\n        Content-Transfer-Encoding: 7bit\n        X-From: Office of the Chairman - Enron Wholesale Services <legalonline-compliance@enron.com>@ENRON <IMCEANOTES-Office+20of+20the+20Chairman+20-+20Enron+20Wholesale+20Services+20+3Clegalonline-compliance+40enron+2Ecom+3E+40ENRON@ENRON.com>\n        X-To: WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@enron.com>\n        X-cc: \n        X-bcc: \n        X-Folder: \\Williams III, Bill (Non-Privileged)\\Bill Williams III\n        X-Origin: Williams-B\n        X-FileName: Williams III, Bill (